In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from local_plot import *
from utils import *


In [9]:
nodes = {1, 2}
output_folder = "/home/xuhao/data/tum_datasets/outputs/d2vins/swarm1"
data_folder="/home/xuhao/data/tum_datasets/"
dte = 40
paths_gt, t0 = read_paths(data_folder, nodes, prefix="groundtruth")
paths, _ = read_paths(output_folder, nodes, t0=t0)
display(plot_fused_err(nodes, paths, paths_gt,dte=dte, show=False, outlier_thres_yaw=0.1))
display(plot_relative_pose_err(1, [2], paths, paths_gt,dte=dte, show=False, outlier_thres=0.2, outlier_yaw_thres=0.1))
#plot_fused(nodes, paths, poses_gt=paths_gt)


Drone,Traj. Len,ATE Pos,ATE Att,Cov/m: x,y,z,PGO:ATE Pos,ATE Att
1by1,28.0m,0.035,1.054,3.7e-04,9.6e-05,1.2e-04,nan,nan°
2by1,30.9m,0.032,0.958,1.2e-04,2.1e-04,1.2e-04,nan,nan°


Relative,EST RMSE: Pos,Ang,BIAS: Pos,Ang
1->2,"0.022,0.018,0.013",0.86°,"0.003,+0.006,-0.002",+0.49°
